In [ ]:
%matplotlib qt
# Genera los plots de matplotlib en una ventana externa (lo cuál nos permite hacer zoom)

In [24]:
from sklearn import tree
import pandas as pd
import numpy as np
import MetaTrader5 as mt5
import matplotlib.pyplot as plt

In [25]:
import os
from dotenv import load_dotenv

load_dotenv()

MT5_LOGIN = os.getenv('MT5_LOGIN')
MT5_PWD = os.getenv('MT5_PWD')

mt5.initialize()

mt5.login(int(MT5_LOGIN), MT5_PWD,'MetaQuotes-Demo')

True

In [26]:
from datetime import datetime

from_date = datetime(2015, 1, 1)
to_date = datetime.now()

rates = mt5.copy_rates_range('AAPL', mt5.TIMEFRAME_D1, from_date, to_date)

data = pd.DataFrame(rates)
data = data.drop('real_volume', axis=1)
data['time'] = pd.to_datetime(data['time'], unit='s')
data['log_returns'] = np.log(data['close']/data['close'].shift(1))
data['fwd_returns'] = data['log_returns'].shift(-1)
data.dropna(inplace=True)
data.set_index('time', inplace=True)

# Normalización ¿?

data

,open,high,low,close,tick_volume,spread,log_returns,fwd_returns
time,,,,,,,,
2015-01-05,27.07,27.16,26.35,26.56,244973,0,-0.028579,0.000000
2015-01-06,26.64,26.86,26.16,26.56,290053,0,0.000000,0.014206
2015-01-07,26.80,27.05,26.67,26.94,173284,0,0.014206,0.037520
2015-01-08,27.31,28.04,27.18,27.97,241731,0,0.037520,0.001072
2015-01-09,28.17,28.31,27.55,28.00,230227,0,0.001072,-0.024585
...,...,...,...,...,...,...,...,...
2025-04-29,208.69,212.24,208.37,211.21,64438,1,0.005079,0.006089
2025-04-30,209.30,213.54,206.67,212.50,79081,1,0.006089,0.003851
2025-05-01,209.01,214.55,208.90,213.32,96456,1,0.003851,-0.038078


In [38]:
X = data[['open', 'high', 'low', 'close', 'tick_volume', 'spread', 'log_returns']]
y = data['fwd_returns']

model = tree.DecisionTreeRegressor(max_depth=50, max_leaf_nodes=50)
model.fit(X,y)
data['prediction'] = model.predict(X)

In [28]:
plt.figure(figsize=(30,10))

tree.plot_tree(model, feature_names=['open', 'high', 'low','close',
                                      'tick_volume', 'spread', 'log_returns'],
                                      filled=True ,max_depth=5)
plt.show()

In [ ]:
def display_plot(data, price_column = 'close', prediction_column = 'prediction'):
    plt.figure(figsize=(30,10))
    plt.plot(data.index, data[price_column], color='black')

    signal_change = np.sign(data[prediction_column].diff().fillna(0)) != 0
    longs = (data[prediction_column] > 0) & signal_change
    shorts = (data[prediction_column] < 0) & signal_change

    plt.scatter(data.index[longs], data[price_column][longs], color='green', label='Long', marker='^', alpha=1, edgecolor='black', linewidth=0.5)
    plt.scatter(data.index[shorts], data[price_column][shorts], color='red', label='Short', marker='v', alpha=1, edgecolor='black', linewidth=0.5)

    start_long = None
    start_short = None

    for i in range(1, len(data)):
        if data[prediction_column].iloc[i-1] <= 0 and data[prediction_column].iloc[i] > 0:
            start_long = i
        elif data[prediction_column].iloc[i-1] > 0 and data[prediction_column].iloc[i] <= 0:
            if start_long is not None:
                plt.axvspan(data.index[start_long], data.index[i], color='green', alpha=0.3)
                start_long = None

        if data[prediction_column].iloc[i-1] >= 0 and data[prediction_column].iloc[i] < 0:
            start_short = i
        elif data[prediction_column].iloc[i-1] < 0 and data[prediction_column].iloc[i] >= 0:
            if start_short is not None:
                plt.axvspan(data.index[start_short], data.index[i], color='red', alpha=0.3)
                start_short = None

    plt.xlabel('Date', fontsize=14)
    plt.ylabel('Price', fontsize=14)
    plt.title('Adjusted Price with Long and Short Signals', fontsize=16)
    plt.legend(loc='upper left', fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.show()

In [39]:
display_plot(data)

In [52]:
test_size = 500

X = data[['open', 'high', 'low', 'close', 'tick_volume', 'spread', 'log_returns']].copy()
y = data['fwd_returns'].copy()

X_test = X.iloc[-test_size:]
y_test = y.iloc[-test_size:]

X_train = X.iloc[:-test_size]
y_train = y.iloc[:-test_size]

model = tree.DecisionTreeRegressor(max_depth=50, max_leaf_nodes=50)
model.fit(X_train, y_train)

data['prediction'] = model.predict(X)

display_plot(data)

In [ ]:
plt.figure(figsize=(10, 6))

plt.plot(np.cumsum(data['fwd_returns']* np.where(data['prediction'] > 0, 1, -1)), 
         label='Cumulative Return', color='b', linewidth=2)

plt.title('Cumulative Forward Returns Based on Predictions', fontsize=14, fontweight="bold") 
plt.xlabel('Time', fontsize=12)
plt.ylabel('Cumulative Returns', fontsize=12)
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.legend (loc='upper left', fontsize=12)


cum_returns = np.cumsum(data['fwd_returns']* np.where(data['prediction'] > 0, 1, -1)) 
max_return= np.max(cum_returns)
min_return = np.min(cum_returns)


plt.annotate(f'Max: {max_return: .2f}', xy=(np.argmax(cum_returns), max_return), xytext=(np.argmax(cum_returns), max_return + 1), arrowprops=dict (facecolor='green', shrink=0.05), fontsize=10)
plt.annotate(f'Min: {min_return: .2f}', xy=(np.argmin (cum_returns), min_return), xytext=(np.argmin (cum_returns), min_return - 1), arrowprops=dict(facecolor='red', shrink=0.05), fontsize=10)
plt.tight_layout()
plt.show()